# 🧪 评估框架 (Evaluation Framework)

> 为不同版本的 Agent 建立完整的性能、质量和成本基准

**目的**: 生成符合 `comparison_report.ipynb` 要求的基准数据

**评估维度**:
1. **性能**: 端到端耗时、LLM响应、数据获取、指标计算
2. **质量**: 格式正确率、内容完整率、分析要素覆盖率
3. **成本**: Token使用量、API费用

In [1]:
# 1. 环境配置
import sys
import json
import time
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Any
import numpy as np

# 添加项目根目录
sys.path.append("..")

from agent_logic import run_agent
from core.llm_client import get_response

print("✅ 模块导入成功")
print(f"📅 评估时间: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Matplotlib 中文字体已配置: STHeiti
✅ 模块导入成功
📅 评估时间: 2025-12-14 23:41:38


## 📋 测试用例集

In [2]:
# 2. 定义测试用例
TEST_CASES = [
    {
        "query": "分析茅台最近两个月的走势",
        "type": "relative_date",
        "expected_elements": ["趋势", "MA", "MACD", "图表"]
    },
    {
        "query": "看看平安银行今年的表现",
        "type": "year_to_date",
        "expected_elements": ["年初", "涨跌", "指标"]
    },
    {
        "query": "分析 600519 和 000858 的 RSI",
        "type": "multi_symbol",
        "expected_elements": ["RSI", "对比"]
    },
    {
        "query": "帮我查一下沪深300ETF的技术指标",
        "type": "etf_analysis",
        "expected_elements": ["ETF", "MACD", "RSI"]
    },
]

print(f"📝 测试用例数: {len(TEST_CASES)}")
for i, case in enumerate(TEST_CASES, 1):
    print(f"  {i}. [{case['type']}] {case['query']}")

📝 测试用例数: 4
  1. [relative_date] 分析茅台最近两个月的走势
  2. [year_to_date] 看看平安银行今年的表现
  3. [multi_symbol] 分析 600519 和 000858 的 RSI
  4. [etf_analysis] 帮我查一下沪深300ETF的技术指标


## 🎯 质量评估函数

In [3]:
# 3. 质量评估逻辑
def evaluate_quality(result: Dict[str, Any], expected_elements: List[str]) -> Dict[str, float]:
    """
    评估 Agent 输出的质量
    
    Returns:
        - format_correct: 格式是否正确（有final_answer）
        - content_complete: 内容完整性（包含期望元素的比例）
        - has_trend_analysis: 是否有趋势分析
        - has_indicator_analysis: 是否有指标分析
        - has_chart: 是否生成图表
    """
    final_answer = result.get("final_answer", "")
    
    # 1. 格式正确性（是否成功返回final_answer）
    format_correct = 1.0 if result.get("success") and final_answer else 0.0
    
    # 2. 内容完整性（包含期望元素的比例）
    if final_answer:
        matched_elements = sum(1 for elem in expected_elements if elem in final_answer)
        content_complete = matched_elements / len(expected_elements) if expected_elements else 0.0
    else:
        content_complete = 0.0
    
    # 3. 分析要素检查
    has_trend_analysis = 1.0 if any(word in final_answer for word in ["上涨", "下跌", "震荡", "趋势"]) else 0.0
    has_indicator_analysis = 1.0 if any(word in final_answer for word in ["MA", "MACD", "RSI", "金叉", "死叉"]) else 0.0
    has_chart = 1.0 if "chart_path" in result and result["chart_path"] else 0.0
    
    return {
        "format_correct": format_correct,
        "content_complete": content_complete,
        "has_trend_analysis": has_trend_analysis,
        "has_indicator_analysis": has_indicator_analysis,
        "has_chart": has_chart,
    }

# 测试评估函数
mock_result = {
    "success": True,
    "final_answer": "茅台近期呈震荡上涨趋势，MA5上穿MA20形成金叉，MACD柱状图转正",
    "chart_path": "outputs/test.png"
}
print("\n🧪 质量评估测试:")
print(evaluate_quality(mock_result, ["趋势", "MA", "MACD"]))


🧪 质量评估测试:
{'format_correct': 1.0, 'content_complete': 1.0, 'has_trend_analysis': 1.0, 'has_indicator_analysis': 1.0, 'has_chart': 1.0}


## 💰 成本估算函数

In [4]:
# 4. Token 成本估算
def estimate_cost(total_tokens: int, model: str = "gpt-4o-mini") -> float:
    """
    估算 API 调用成本（人民币）
    
    价格参考（2024年12月）：
    - gpt-4o-mini: $0.15/1M input + $0.60/1M output (约 ¥1/1M + ¥4/1M)
    - qwen-max: ¥0.12/1K tokens
    - glm-4: ¥0.10/1K tokens
    """
    pricing = {
        "gpt-4o-mini": 2.5 / 1_000_000,  # 平均 ¥2.5/1M tokens
        "gpt-4o": 15 / 1_000_000,
        "qwen-max": 0.12 / 1000,
        "glm-4": 0.10 / 1000,
        "deepseek-chat": 0.014 / 1000,
    }
    
    # 提取模型前缀（如 qwen3-max -> qwen-max）
    model_key = None
    for key in pricing:
        if key in model.lower():
            model_key = key
            break
    
    if not model_key:
        model_key = "gpt-4o-mini"  # 默认
    
    return total_tokens * pricing[model_key]

print("💰 成本估算测试:")
print(f"  2500 tokens (gpt-4o-mini): ¥{estimate_cost(2500, 'gpt-4o-mini'):.4f}")
print(f"  2500 tokens (qwen-max): ¥{estimate_cost(2500, 'qwen-max'):.4f}")

💰 成本估算测试:
  2500 tokens (gpt-4o-mini): ¥0.0063
  2500 tokens (qwen-max): ¥0.3000


## 🚀 运行完整评估

In [5]:
# 5. 运行完整评估
def run_evaluation(
    version: str,
    test_cases: List[Dict],
    model: str = "gpt-4o-mini",
    verbose: bool = True
) -> Dict[str, Any]:
    """
    对指定版本进行完整评估
    """
    print(f"\n{'='*60}")
    print(f"🧪 评估版本: {version}")
    print(f"🤖 测试模型: {model}")
    print(f"{'='*60}\n")
    
    results = []
    performance_metrics = []
    quality_metrics = []
    cost_metrics = []
    
    for i, case in enumerate(test_cases, 1):
        print(f"[{i}/{len(test_cases)}] 测试: {case['query']}")
        
        try:
            # 运行 Agent（带性能监控）
            start_time = time.time()
            result = run_agent(
                case["query"],
                model=model,
                verbose=False  # 关闭详细输出以加速
            )
            end_time = time.time()
            
            duration = end_time - start_time
            
            # 性能指标
            performance_metrics.append({
                "end_to_end_time": duration,
                "llm_response_time": result.get("llm_time", duration * 0.6),  # 估算
                "data_fetching_time": result.get("data_time", duration * 0.3),  # 估算
            })
            
            # 质量指标
            quality = evaluate_quality(result, case["expected_elements"])
            quality_metrics.append(quality)
            
            # 成本指标
            total_tokens = result.get("total_tokens", 2500)  # 默认估算值
            cost = estimate_cost(total_tokens, model)
            cost_metrics.append({
                "total_tokens": total_tokens,
                "cost_cny": cost
            })
            
            status = "✅" if result.get("success") else "❌"
            print(f"  {status} 耗时: {duration:.2f}s | Token: {total_tokens} | 成本: ¥{cost:.4f}")
            
        except Exception as e:
            print(f"  ❌ 失败: {str(e)}")
            # 记录失败案例
            performance_metrics.append({"end_to_end_time": 0, "llm_response_time": 0, "data_fetching_time": 0})
            quality_metrics.append({k: 0.0 for k in ["format_correct", "content_complete", "has_trend_analysis", "has_indicator_analysis", "has_chart"]})
            cost_metrics.append({"total_tokens": 0, "cost_cny": 0})
    
    # 汇总统计
    benchmark_data = {
        "version": version,
        "timestamp": datetime.now().isoformat(),
        "model": model,
        "test_cases_count": len(test_cases),
        "performance": {
            "end_to_end_avg_time": np.mean([m["end_to_end_time"] for m in performance_metrics]),
            "llm_avg_response_time": np.mean([m["llm_response_time"] for m in performance_metrics]),
            "data_fetching_avg_time": np.mean([m["data_fetching_time"] for m in performance_metrics]),
        },
        "quality": {
            "format_correct_rate": np.mean([m["format_correct"] for m in quality_metrics]),
            "content_complete_rate": np.mean([m["content_complete"] for m in quality_metrics]),
            "has_trend_analysis_rate": np.mean([m["has_trend_analysis"] for m in quality_metrics]),
            "has_indicator_analysis_rate": np.mean([m["has_indicator_analysis"] for m in quality_metrics]),
            "has_chart_rate": np.mean([m["has_chart"] for m in quality_metrics]),
        },
        "cost": {
            "avg_total_tokens": int(np.mean([m["total_tokens"] for m in cost_metrics])),
            "total_cost_cny": np.sum([m["cost_cny"] for m in cost_metrics]),
            "avg_cost_per_query_cny": np.mean([m["cost_cny"] for m in cost_metrics]),
        }
    }
    
    return benchmark_data

print("✅ 评估函数已定义")

✅ 评估函数已定义


## 📊 执行评估并保存结果

In [6]:
# 6. 运行评估（当前版本 v1.2.0）
VERSION = "v1.2.0"
MODEL = "gpt-4o-mini"  # 可修改为其他模型

print("⚠️  注意: 此过程将调用真实的 LLM API，可能需要 1-2 分钟")
print("确认继续请运行此单元格...\n")

benchmark_result = run_evaluation(
    version=VERSION,
    test_cases=TEST_CASES,
    model=MODEL,
    verbose=True
)

# 打印汇总
print("\n" + "="*60)
print("📊 评估结果汇总")
print("="*60)
print(f"\n⚡ 性能:")
print(f"  端到端平均时间: {benchmark_result['performance']['end_to_end_avg_time']:.2f}s")
print(f"  LLM 响应时间: {benchmark_result['performance']['llm_avg_response_time']:.2f}s")

print(f"\n✅ 质量:")
print(f"  格式正确率: {benchmark_result['quality']['format_correct_rate']*100:.1f}%")
print(f"  内容完整率: {benchmark_result['quality']['content_complete_rate']*100:.1f}%")

print(f"\n💰 成本:")
print(f"  平均 Token: {benchmark_result['cost']['avg_total_tokens']}")
print(f"  总成本: ¥{benchmark_result['cost']['total_cost_cny']:.4f}")

⚠️  注意: 此过程将调用真实的 LLM API，可能需要 1-2 分钟
确认继续请运行此单元格...


🧪 评估版本: v1.2.0
🤖 测试模型: gpt-4o-mini

[1/4] 测试: 分析茅台最近两个月的走势
  ✅ 耗时: 0.00s | Token: 2500 | 成本: ¥0.0063
[2/4] 测试: 看看平安银行今年的表现
  ✅ 耗时: 0.00s | Token: 2500 | 成本: ¥0.0063
[3/4] 测试: 分析 600519 和 000858 的 RSI
  ✅ 耗时: 0.00s | Token: 2500 | 成本: ¥0.0063
[4/4] 测试: 帮我查一下沪深300ETF的技术指标
  ✅ 耗时: 0.00s | Token: 2500 | 成本: ¥0.0063

📊 评估结果汇总

⚡ 性能:
  端到端平均时间: 0.00s
  LLM 响应时间: 0.00s

✅ 质量:
  格式正确率: 100.0%
  内容完整率: 0.0%

💰 成本:
  平均 Token: 2500
  总成本: ¥0.0250


In [7]:
# 7. 保存基准数据
output_path = f"benchmarks/{VERSION}_baseline.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(benchmark_result, f, indent=2, ensure_ascii=False)

print(f"✅ 基准数据已保存: {output_path}")
print("\n📝 下一步:")
print("  1. 运行优化实验（optimize_llm_prompts.ipynb）")
print("  2. 生成新版本的基准数据（修改 VERSION 后重新运行此 Notebook）")
print("  3. 使用 comparison_report.ipynb 对比多版本数据")

✅ 基准数据已保存: benchmarks/v1.2.0_baseline.json

📝 下一步:
  1. 运行优化实验（optimize_llm_prompts.ipynb）
  2. 生成新版本的基准数据（修改 VERSION 后重新运行此 Notebook）
  3. 使用 comparison_report.ipynb 对比多版本数据
